## Generate AWQ Config Files

In [3]:
import json
import os
import itertools

In [54]:
dir_name = 'retrieval_configs'
os.makedirs(f"./{dir_name}", exist_ok=True)

In [12]:
model_part2modules = {
    'self_attn' : ['self_attn', 'self_attn_output'],
    'fc': ['fc1', 'fc2'],
    'cross_attn': ['cross_attn', 'cross_attn_output'],
    'img_ff' : ['intermediate_query', 'output_query'],
    'txt_ff' : ['intermediate_txt', 'output_txt']
}

#### Info-retrieval Configs

In [50]:
# bit_options = [8,6,4,2]
bit_options = [4,2]
vit_model_parts = ['self_attn', 'fc']

bit_widths = [bit_options for _ in range(len(vit_model_parts))] 
vit_bits = list(itertools.product(*bit_widths))

qformer_model_parts = ['self_attn', 'txt_ff', 'img_ff', 'cross_attn']
bit_widths = [bit_options for _ in range(len(qformer_model_parts))]
qformer_bits = list(itertools.product(*bit_widths))

all_bits = list(itertools.product(*[vit_bits, qformer_bits]))
len(all_bits)

64

In [55]:
for i,bits in enumerate(all_bits):

    json_config = {}
    json_config['vit_layers'] = {}
    json_config['qformer_layers'] = {}

    vit_bits, qformer_bits = bits

    # ['self_attn', 'fc']
    for vit_model_part, bit_width in zip(vit_model_parts, vit_bits):
        modules = model_part2modules[vit_model_part]
        for mod in modules:
            json_config['vit_layers'][mod] = bit_width


    # ['self_attn', 'text_ff', 'img_ff', 'cross_attn']
    for qformer_model_part, bit_width in zip(qformer_model_parts, qformer_bits):
        modules = model_part2modules[qformer_model_part]
        for mod in modules:
            json_config['qformer_layers'][mod] = bit_width
        
    filename = os.path.join(dir_name, f'awq_{i}')
    with open(filename, 'w') as f:
        json.dump(json_config, f, indent=2)



{'vit_layers': {'self_attn': 4, 'self_attn_output': 4, 'fc1': 4, 'fc2': 4}, 'qformer_layers': {'self_attn': 4, 'self_attn_output': 4, 'intermediate_txt': 4, 'output_txt': 4, 'intermediate_query': 4, 'output_query': 4, 'cross_attn': 4, 'cross_attn_output': 4}}
{'vit_layers': {'self_attn': 4, 'self_attn_output': 4, 'fc1': 4, 'fc2': 4}, 'qformer_layers': {'self_attn': 4, 'self_attn_output': 4, 'intermediate_txt': 4, 'output_txt': 4, 'intermediate_query': 4, 'output_query': 4, 'cross_attn': 2, 'cross_attn_output': 2}}
{'vit_layers': {'self_attn': 4, 'self_attn_output': 4, 'fc1': 4, 'fc2': 4}, 'qformer_layers': {'self_attn': 4, 'self_attn_output': 4, 'intermediate_txt': 4, 'output_txt': 4, 'intermediate_query': 2, 'output_query': 2, 'cross_attn': 4, 'cross_attn_output': 4}}
{'vit_layers': {'self_attn': 4, 'self_attn_output': 4, 'fc1': 4, 'fc2': 4}, 'qformer_layers': {'self_attn': 4, 'self_attn_output': 4, 'intermediate_txt': 4, 'output_txt': 4, 'intermediate_query': 2, 'output_query': 2, 'c

In [38]:
len(all_configs)

64

In [57]:
test_config = json.load(open('/nfshomes/vla/low_bit_vision/mmq/awq/retrieval_configs/awq_58'))

In [58]:
test_config

{'vit_layers': {'self_attn': 2, 'self_attn_output': 2, 'fc1': 2, 'fc2': 2},
 'qformer_layers': {'self_attn': 2,
  'self_attn_output': 2,
  'intermediate_txt': 4,
  'output_txt': 4,
  'intermediate_query': 2,
  'output_query': 2,
  'cross_attn': 4,
  'cross_attn_output': 4}}

#### Caption Generation Configs

In [4]:
dir_name = 'captioning_configs'
os.makedirs(f"./{dir_name}", exist_ok=True)

In [6]:
# bit_options = [8,6,4,2]
bit_options = [4,2]

vit_model_parts = ['self_attn', 'fc']
bit_widths = [bit_options for _ in range(len(vit_model_parts))] 
vit_bits = list(itertools.product(*bit_widths))

qformer_model_parts = ['self_attn', 'img_ff', 'cross_attn']
bit_widths = [bit_options for _ in range(len(qformer_model_parts))]
qformer_bits = list(itertools.product(*bit_widths))

llm_model_parts = ['self_attn', 'fc']
bit_widths = [bit_options for _ in range(len(llm_model_parts))]
llm_bits = list(itertools.product(*bit_widths))


all_bits = list(itertools.product(*[vit_bits, qformer_bits, llm_bits]))
len(all_bits)

128

In [13]:
for i,bits in enumerate(all_bits):

    json_config = {}
    json_config['vit_layers'] = {}
    json_config['qformer_layers'] = {}
    json_config['llm_layers'] = {}

    vit_bits, qformer_bits, llm_bits = bits

    # ['self_attn', 'fc']
    for vit_model_part, bit_width in zip(vit_model_parts, vit_bits):
        modules = model_part2modules[vit_model_part]
        for mod in modules:
            json_config['vit_layers'][mod] = bit_width


    # ['self_attn', 'text_ff', 'img_ff', 'cross_attn']
    for qformer_model_part, bit_width in zip(qformer_model_parts, qformer_bits):
        modules = model_part2modules[qformer_model_part]
        for mod in modules:
            json_config['qformer_layers'][mod] = bit_width

    # ['self_attn', 'fc']
    for llm_model_part, bit_width in zip(llm_model_parts, llm_bits):
        modules = model_part2modules[llm_model_part]
        for mod in modules:
            json_config['llm_layers'][mod] = bit_width
        
    filename = os.path.join(dir_name, f'awq_{i}')
    with open(filename, 'w') as f:
        json.dump(json_config, f, indent=2)



In [14]:
test_config = json.load(open('/nfshomes/vla/low_bit_vision/mmq/awq/captioning_configs/awq_55'))

In [15]:
test_config

{'vit_layers': {'self_attn': 4, 'self_attn_output': 4, 'fc1': 2, 'fc2': 2},
 'qformer_layers': {'self_attn': 2,
  'self_attn_output': 2,
  'intermediate_query': 4,
  'output_query': 4,
  'cross_attn': 2,
  'cross_attn_output': 2},
 'llm_layers': {'self_attn': 2, 'self_attn_output': 2, 'fc1': 2, 'fc2': 2}}